# Complete DIY Home Improvement Guide Generator

Generate comprehensive DIY guides with:
- **Step-by-step instructions** with detailed explanations
- **Visual guidance** with AI-generated images for each step
- **Safety protocols** and precautions
- **Tools & materials** with specifications
- **Product recommendations** with pricing and comparison
- **Purchase links** from major retailers (Home Depot, Lowe's, Amazon)
- **Cost breakdown** and budget planning

**Powered by:** Google Gemini 2.5 Flash & Imagen 4.0

## 📋 Prerequisites & Setup

**Required:**
1. Google AI Studio API key ([Get one here](https://aistudio.google.com/app/apikey))
2. Python packages: `google-genai`, `pillow`, `requests`

**Set your API key:**
```powershell
$env:GOOGLE_API_KEY = "your-api-key-here"
```

In [87]:
# Uncomment to install required packages
# %pip install --quiet google-genai pillow requests

## 🔧 Import Libraries & Configure

In [88]:
import os
import json
import re
import textwrap
import pathlib
import base64
import io
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any
from datetime import datetime

import google.generativeai as genai
from google import genai as genai_client
from google.genai import types
from PIL import Image
from IPython.display import display, Markdown, HTML

# Verify API key
API_KEY = os.getenv('GOOGLE_API_KEY')
if not API_KEY:
    raise RuntimeError(
        '❌ Missing GOOGLE_API_KEY environment variable.\n'
        'Set it in PowerShell: $env:GOOGLE_API_KEY = "your-key"'
    )

# Configure Gemini
genai.configure(api_key=API_KEY)
imagen_client = genai_client.Client(api_key=API_KEY)

# Model configuration
TEXT_MODEL_NAME = 'gemini-2.5-flash'
IMAGE_MODEL_NAME = 'imagen-4.0-generate-001'

# Create exports directory
EXPORTS_DIR = pathlib.Path('../exports/diy_guides')
EXPORTS_DIR.mkdir(parents=True, exist_ok=True)

print('✅ Configuration loaded successfully!')
print(f'📝 Text Model: {TEXT_MODEL_NAME}')
print(f'🎨 Image Model: {IMAGE_MODEL_NAME}')
print(f'📁 Export Directory: {EXPORTS_DIR.absolute()}')

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


✅ Configuration loaded successfully!
📝 Text Model: gemini-2.5-flash
🎨 Image Model: imagen-4.0-generate-001
📁 Export Directory: c:\Users\ramma\Documents\augment-projects\augment\notebooks\..\exports\diy_guides


## 📚 Data Structures

In [89]:
@dataclass
class Product:
    """Product information with pricing and purchase options."""
    name: str
    category: str  # 'material' or 'tool'
    description: str
    specifications: Optional[str] = None
    estimated_price: Optional[str] = None
    retailers: List[Dict[str, str]] = field(default_factory=list)  # [{'name': 'Home Depot', 'price': '$XX.XX', 'url': '...'}]
    alternatives: Optional[List[str]] = None
    rental_option: Optional[str] = None  # For tools that can be rented

@dataclass
class SafetyGuideline:
    """Safety information with warnings and requirements."""
    category: str  # 'warning', 'protective_equipment', 'precaution'
    description: str
    importance: str  # 'critical', 'high', 'medium'

@dataclass
class GuideStep:
    """Single step in the DIY guide."""
    step_number: int
    title: str
    description: str
    detailed_instructions: List[str]
    tips: Optional[List[str]] = None
    warnings: Optional[List[str]] = None
    common_mistakes: Optional[List[str]] = None  # Common mistakes to avoid
    quality_checkpoints: Optional[List[str]] = None  # Quality checks for this step
    image_prompt: Optional[str] = None
    image_path: Optional[str] = None
    required_products: Optional[List[str]] = None  # Product names referenced

@dataclass
class CostBreakdown:
    """Project cost analysis."""
    materials_cost: Dict[str, float]  # {product_name: price}
    tools_cost: Dict[str, float]
    total_materials: float
    total_tools: float
    total_project: float
    cost_notes: Optional[List[str]] = None

@dataclass
class CompleteDIYGuide:
    """Complete DIY guide with all information."""
    # Basic Info
    topic: str
    difficulty: str
    duration_estimate: str
    overview: str
    
    # Safety
    safety_guidelines: List[SafetyGuideline]
    
    # Products
    materials: List[Product]
    tools: List[Product]
    
    # Instructions
    preparation_steps: List[str]
    steps: List[GuideStep]
    
    # Cost
    cost_breakdown: CostBreakdown
    
    # Additional Info
    cleanup_instructions: Optional[str] = None
    maintenance_tips: Optional[List[str]] = None
    troubleshooting: Optional[Dict[str, str]] = None
    permits_required: Optional[str] = None
    
    # Metadata
    generated_date: str = field(default_factory=lambda: datetime.now().strftime('%Y-%m-%d'))

print('✅ Data structures defined')

✅ Data structures defined


## 🤖 Initialize AI Models

In [90]:
# Initialize text generation model
text_model = genai.GenerativeModel(TEXT_MODEL_NAME)
print(f'✅ Text model initialized: {TEXT_MODEL_NAME}')
print(f'✅ Image generation client ready: {IMAGE_MODEL_NAME}')

✅ Text model initialized: gemini-2.5-flash
✅ Image generation client ready: imagen-4.0-generate-001


## 🎯 Guide Generation Functions

In [91]:
def build_comprehensive_guide_prompt(topic: str, budget_range: str = 'moderate') -> str:
    """Build a comprehensive prompt for complete DIY guide generation."""
    
    system_instructions = textwrap.dedent(f'''
    You are an expert DIY home improvement consultant with 20+ years of experience in
    construction, remodeling, and teaching homeowners. Create a COMPLETE, professional
    DIY guide that covers every aspect of the project.
    
    Project Budget Range: {budget_range}
    
    REQUIREMENTS:
    
    1. OVERVIEW & PLANNING:
       - Clear project description with benefits
       - Realistic difficulty assessment
       - Accurate time estimate (prep + work + cleanup)
       - Required skill level
    
    2. SAFETY GUIDELINES:
       - Critical warnings (electrical, structural, chemical hazards)
       - Required protective equipment (specific items)
       - Important precautions
       - When to call a professional
    
    3. MATERIALS LIST:
       - Specific products with brand recommendations
       - Exact quantities needed
       - Specifications (sizes, ratings, grades)
       - Estimated prices (realistic US prices)
       - Alternative options (budget/premium)
       - Where to buy (Home Depot, Lowe\'s, Amazon product names)
    
    4. TOOLS LIST:
       - All required tools (basic and specialized)
       - Tool specifications
       - Purchase vs rental recommendations
       - Estimated costs
       - Alternative tools
    
    5. STEP-BY-STEP INSTRUCTIONS:
       - Detailed preparation steps
       - Numbered main steps with clear titles
       - Multiple sub-instructions per step
       - Pro tips and tricks
       - Common mistakes to avoid
       - Quality checkpoints
    
    6. COST BREAKDOWN:
       - Individual material costs
       - Individual tool costs
       - Total materials cost
       - Total tools cost (one-time)
       - Overall project total
       - Cost-saving tips
    
    7. FINISHING TOUCHES:
       - Cleanup and disposal instructions
       - Maintenance recommendations
       - Troubleshooting common issues
       - Permit/inspection requirements if any
    
    FORMAT REQUIREMENTS:
    - Use US measurements only
    - Include real product names from major US retailers
    - Provide realistic price ranges
    - Be specific with quantities (e.g., "2 boxes of 1-1/4 inch screws (100ct)")
    - Include model numbers or specific product lines when relevant
    - Suggest both in-store and online purchase options
    ''')
    
    json_schema = textwrap.dedent('''
    Return ONLY valid JSON with this structure:
    {
      "topic": "string",
      "difficulty": "Beginner|Intermediate|Advanced",
      "duration_estimate": "string",
      "overview": "string (2-3 paragraphs)",
      "permits_required": "string or null",
      
      "safety_guidelines": [
        {
          "category": "warning|protective_equipment|precaution",
          "description": "string",
          "importance": "critical|high|medium"
        }
      ],
      
      "materials": [
        {
          "name": "string",
          "category": "material",
          "description": "string (what it\'s for)",
          "specifications": "string (size, quantity, grade)",
          "estimated_price": "string ($X.XX - $Y.YY)",
          "retailers": [
            {
              "name": "Home Depot|Lowe\'s|Amazon|etc",
              "price": "$XX.XX",
              "product_name": "specific product name or brand",
              "notes": "optional availability notes"
            }
          ],
          "alternatives": ["string"] (optional)
        }
      ],
      
      "tools": [
        {
          "name": "string",
          "category": "tool",
          "description": "string",
          "specifications": "string",
          "estimated_price": "string",
          "retailers": [...],
          "alternatives": ["string"] (optional),
          "rental_option": "string (if applicable)"
        }
      ],
      
      "preparation_steps": ["string"],
      
      "steps": [
        {
          "step_number": 1,
          "title": "string",
          "description": "string (brief overview)",
          "detailed_instructions": ["string"],
          "tips": ["string"] (optional),
          "warnings": ["string"] (optional),
          "image_prompt": "string (detailed description for AI image generation)",
          "required_products": ["string"] (product names from materials/tools)
        }
      ],
      
      "cost_breakdown": {
        "materials_cost": {"product_name": 25.99},
        "tools_cost": {"tool_name": 49.99},
        "total_materials": 150.00,
        "total_tools": 200.00,
        "total_project": 350.00,
        "cost_notes": ["string"] (optional)
      },
      
      "cleanup_instructions": "string",
      "maintenance_tips": ["string"],
      "troubleshooting": {"problem": "solution"}
    }
    ''')
    
    return f"{system_instructions}\n\nTopic: {topic}\n\n{json_schema}"

print('✅ Prompt builder defined')

✅ Prompt builder defined


In [92]:
def parse_guide_response(response_text: str, topic: str) -> CompleteDIYGuide:
    """Parse the JSON response and create a CompleteDIYGuide object."""
    
    # Clean response text - handle markdown code blocks
    response_text = response_text.strip()
    
    # Remove markdown code blocks if present
    if response_text.startswith('```'):
        # Find the first opening ``` and remove it with any language identifier
        response_text = re.sub(r'^```[a-z]*\s*\n', '', response_text)
        # Remove the closing ```
        response_text = re.sub(r'\n```\s*$', '', response_text)
        response_text = response_text.strip()
    
    # Try to parse JSON
    data = None
    try:
        data = json.loads(response_text)
        print(f"✅ Successfully parsed JSON response")
    except json.JSONDecodeError as e:
        print(f"⚠️ Initial JSON parsing failed: {str(e)}")
        print(f"Error at line {e.lineno}, column {e.colno}, char {e.pos}")
        
        # Check if JSON is truncated
        if not response_text.rstrip().endswith('}'):
            print(f"⚠️ JSON appears to be truncated - attempting to fix...")
            
            # Try to close truncated strings and objects
            response_text = response_text.rstrip()
            
            # If we're in the middle of a string, close it
            # Count quotes to see if we're in a string
            quote_count = response_text.count('"') - response_text.count('\\"')
            if quote_count % 2 == 1:  # Odd number means unclosed string
                print("   Closing unclosed string...")
                response_text += '"'
            
            # Count open braces and brackets
            open_braces = response_text.count('{') - response_text.count('}')
            open_brackets = response_text.count('[') - response_text.count(']')
            
            # Close any open arrays
            if open_brackets > 0:
                print(f"   Closing {open_brackets} open array(s)...")
                response_text += ']' * open_brackets
            
            # Close any open objects
            if open_braces > 0:
                print(f"   Closing {open_braces} open object(s)...")
                response_text += '}' * open_braces
            
            # Try parsing again
            try:
                data = json.loads(response_text)
                print(f"✅ Successfully parsed fixed JSON")
            except json.JSONDecodeError as e2:
                print(f"⚠️ Still failed after attempting to fix: {str(e2)}")
        
        # If still no data, try extracting from start
        if data is None:
            print(f"Attempting to extract valid JSON from response...")
            
            # Find the first { and try to parse from there
            start_idx = response_text.find('{')
            if start_idx == -1:
                raise ValueError(f'No JSON object found in response.\n\nFirst 500 chars:\n{response_text[:500]}')
            
            # Try to extract complete JSON by finding matching braces
            depth = 0
            in_string = False
            escape_next = False
            end_idx = -1
            
            for i in range(start_idx, len(response_text)):
                char = response_text[i]
                
                # Handle string escape sequences
                if escape_next:
                    escape_next = False
                    continue
                    
                if char == '\\' and in_string:
                    escape_next = True
                    continue
                
                # Handle strings (don't count braces inside strings)
                if char == '"':
                    in_string = not in_string
                    continue
                
                # Only count braces outside of strings
                if not in_string:
                    if char == '{':
                        depth += 1
                    elif char == '}':
                        depth -= 1
                        if depth == 0:
                            end_idx = i + 1
                            break
            
            if end_idx == -1:
                print(f"⚠️ Could not find matching braces")
                raise ValueError(f'Could not find complete JSON object in response.')
            else:
                json_str = response_text[start_idx:end_idx]
                try:
                    data = json.loads(json_str)
                    print(f"✅ Successfully parsed extracted JSON")
                except json.JSONDecodeError as e3:
                    print(f"⚠️ Extracted JSON parsing failed: {str(e3)}")
                    raise ValueError(f'Could not parse extracted JSON.\n\nError: {str(e3)}')
    
    if data is None:
        raise ValueError("Failed to parse JSON from response")
    
    # Parse safety guidelines
    safety_guidelines = [
        SafetyGuideline(**sg) for sg in data.get('safety_guidelines', [])
    ]
    
    # Parse materials
    materials = [
        Product(**{**m, 'category': 'material'}) for m in data.get('materials', [])
    ]
    
    # Parse tools
    tools = [
        Product(**{**t, 'category': 'tool'}) for t in data.get('tools', [])
    ]
    
    # Parse steps
    steps = [
        GuideStep(**step) for step in data.get('steps', [])
    ]
    
    # Parse cost breakdown
    cost_data = data.get('cost_breakdown', {})
    cost_breakdown = CostBreakdown(
        materials_cost=cost_data.get('materials_cost', {}),
        tools_cost=cost_data.get('tools_cost', {}),
        total_materials=cost_data.get('total_materials', 0.0),
        total_tools=cost_data.get('total_tools', 0.0),
        total_project=cost_data.get('total_project', 0.0),
        cost_notes=cost_data.get('cost_notes')
    )
    
    # Create guide object
    guide = CompleteDIYGuide(
        topic=data.get('topic', topic),
        difficulty=data.get('difficulty', 'Intermediate'),
        duration_estimate=data.get('duration_estimate', 'Varies'),
        overview=data.get('overview', ''),
        safety_guidelines=safety_guidelines,
        materials=materials,
        tools=tools,
        preparation_steps=data.get('preparation_steps', []),
        steps=steps,
        cost_breakdown=cost_breakdown,
        cleanup_instructions=data.get('cleanup_instructions'),
        maintenance_tips=data.get('maintenance_tips'),
        troubleshooting=data.get('troubleshooting'),
        permits_required=data.get('permits_required')
    )
    
    return guide

print('✅ Response parser defined with robust JSON extraction and repair')

✅ Response parser defined with robust JSON extraction and repair


In [93]:
# DEBUG: Test the raw API response
def test_raw_response(topic: str = "How to Install a Ceiling Fan"):
    """Test what we get from the API."""
    print(f'🔍 Testing API for: {topic}\n')
    
    prompt = build_comprehensive_guide_prompt(topic, 'moderate')
    
    response = text_model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            temperature=0.7,
            max_output_tokens=8000,
        )
    )
    
    response_text = response.text if hasattr(response, 'text') else str(response)
    
    print("="*80)
    print("RAW RESPONSE (first 2000 characters):")
    print("="*80)
    print(response_text[:2000])
    print("\n" + "="*80)
    print(f"Response type: {type(response_text)}")
    print(f"Response length: {len(response_text)}")
    print("="*80)
    
    # Try to find JSON
    if response_text.startswith('```'):
        print("\n⚠️ Response starts with markdown code block")
    if '{' in response_text:
        first_brace = response_text.find('{')
        print(f"\n✓ First '{{' found at position {first_brace}")
        print(f"Characters before first brace: {repr(response_text[:first_brace])}")
    else:
        print("\n❌ No '{' found in response!")
    
    return response_text

# Run the test
test_response = test_raw_response()

🔍 Testing API for: How to Install a Ceiling Fan

RAW RESPONSE (first 2000 characters):
```json
{
  "topic": "How to Install a Ceiling Fan",
  "difficulty": "Intermediate",
  "duration_estimate": "3-6 hours (including prep, work, and cleanup)",
  "overview": "Installing a ceiling fan is a fantastic DIY project that can significantly enhance the comfort and aesthetics of almost any room in your home. Not only does it provide a refreshing breeze during warmer months, but many models also include integrated lighting, offering both ambient and task illumination. This project is ideal for improving air circulation, reducing reliance on air conditioning, and adding a stylish focal point to your living space. While it involves basic electrical wiring and working overhead, it's a manageable task for homeowners comfortable with following detailed instructions and adhering to strict safety protocols.\n\nThis comprehensive guide will walk you through every step, from selecting the right fan and en

In [94]:
# DEBUG: Let's test the API response to see what we're getting
def debug_api_response(topic: str):
    """Debug function to see the raw API response."""
    print(f'🔍 Testing API response for: {topic}\n')
    
    prompt = build_comprehensive_guide_prompt(topic, 'moderate')
    
    response = text_model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            temperature=0.7,
            max_output_tokens=8000,
        )
    )
    
    response_text = response.text if hasattr(response, 'text') else str(response)
    
    print("="*80)
    print("RAW RESPONSE (first 2000 characters):")
    print("="*80)
    print(response_text[:2000])
    print("="*80)
    
    return response_text

# Test it now:
test_response = debug_api_response("How to Install a Ceiling Fan")

🔍 Testing API response for: How to Install a Ceiling Fan

RAW RESPONSE (first 2000 characters):
```json
{
  "topic": "How to Install a Ceiling Fan",
  "difficulty": "Intermediate",
  "duration_estimate": "4 to 8 hours (including prep, installation, and cleanup)",
  "overview": "Upgrading your home with a new ceiling fan is a fantastic DIY project that significantly enhances comfort and aesthetic appeal. A ceiling fan provides a gentle breeze to cool down a room during warmer months, allowing you to potentially raise your thermostat settings and save on energy bills. In cooler months, many fans offer a reverse setting to circulate warm air trapped near the ceiling, improving heating efficiency.\n\nThis guide will walk you through the process of safely and correctly installing a ceiling fan, whether you're replacing an old fixture or adding a fan to a pre-wired location. We'll cover everything from ensuring proper electrical support and structural integrity to making secure wire connecti

In [95]:
def generate_complete_guide(topic: str, budget_range: str = 'moderate', generate_images: bool = True) -> CompleteDIYGuide:
    """Generate a complete DIY guide with all information."""
    
    print(f'🔄 Generating complete DIY guide for: {topic}')
    print(f'💰 Budget range: {budget_range}')
    print(f'🎨 Generate images: {generate_images}')
    print()
    
    # Generate guide content
    print('📝 Step 1: Generating guide content...')
    prompt = build_comprehensive_guide_prompt(topic, budget_range)
    
    # Use higher max_output_tokens to avoid truncation
    response = text_model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            temperature=0.7,
            max_output_tokens=16000,  # Increased from 8000
            response_mime_type="application/json",  # Request JSON directly
        )
    )
    
    response_text = response.text if hasattr(response, 'text') else str(response)
    
    # Check if response looks truncated
    if not response_text.rstrip().endswith('}'):
        print(f"⚠️ Warning: Response may be truncated (doesn't end with '}}')")
        print(f"Response length: {len(response_text)} characters")
        print(f"Last 200 characters: ...{response_text[-200:]}")
    
    guide = parse_guide_response(response_text, topic)
    print(f'✅ Generated {len(guide.steps)} steps with {len(guide.materials)} materials and {len(guide.tools)} tools')
    
    # Generate images if requested
    if generate_images and len(guide.steps) > 0:
        print(f'\n🎨 Step 2: Generating images for {len(guide.steps)} steps...')
        
        # Create images directory
        images_dir = EXPORTS_DIR / 'images' / re.sub(r'[^a-z0-9]+', '_', topic.lower())
        images_dir.mkdir(parents=True, exist_ok=True)
        
        for i, step in enumerate(guide.steps, 1):
            if step.image_prompt:
                try:
                    print(f'  Generating image {i}/{len(guide.steps)}: {step.title[:50]}...')
                    
                    # Generate image
                    image_response = imagen_client.models.generate_images(
                        model=IMAGE_MODEL_NAME,
                        prompt=step.image_prompt,
                        config=types.GenerateImagesConfig(
                            number_of_images=1,
                            aspect_ratio='16:9',
                            safety_filter_level='block_some',
                            person_generation='allow_adult'
                        )
                    )
                    
                    if image_response.generated_images:
                        # Save image
                        img_data = image_response.generated_images[0].image.image_bytes
                        img = Image.open(io.BytesIO(img_data))
                        
                        img_filename = f'step_{i:02d}_{re.sub(r"[^a-z0-9]+", "_", step.title.lower())}.png'
                        img_path = images_dir / img_filename
                        img.save(img_path)
                        
                        step.image_path = str(img_path)
                        print(f'    ✅ Saved: {img_filename}')
                    
                except Exception as e:
                    print(f'    ⚠️ Failed to generate image: {str(e)[:100]}')
        
        print('✅ Image generation complete')
    
    print(f'\n✅ Complete guide generated successfully!')
    return guide

print('✅ Guide generator defined')

✅ Guide generator defined


## 📄 Export Functions

In [96]:
def format_currency(amount: float) -> str:
    """Format a number as US currency."""
    return f"${amount:,.2f}"

def export_guide_to_markdown(guide: CompleteDIYGuide, filename: Optional[str] = None) -> str:
    """Export guide to a beautiful markdown file."""
    
    if not filename:
        filename = f"{re.sub(r'[^a-z0-9]+', '_', guide.topic.lower())}.md"
    
    filepath = EXPORTS_DIR / filename
    
    # Build markdown content
    md_lines = []
    
    # Header
    md_lines.extend([
        f"# {guide.topic}",
        "",
        f"**Difficulty:** {guide.difficulty} | ",
        f"**Time Required:** {guide.duration_estimate} | ",
        f"**Estimated Cost:** {format_currency(guide.cost_breakdown.total_project)}",
        "",
        f"*Generated: {guide.generated_date}*",
        "",
        "---",
        ""
    ])
    
    # Overview
    md_lines.extend([
        "## 📋 Overview",
        "",
        guide.overview,
        ""
    ])
    
    # Permits
    if guide.permits_required:
        md_lines.extend([
            "### 📜 Permits & Inspections",
            "",
            guide.permits_required,
            ""
        ])
    
    # Safety Guidelines
    md_lines.extend([
        "## ⚠️ Safety Guidelines",
        ""
    ])
    
    # Group by importance
    critical = [sg for sg in guide.safety_guidelines if sg.importance == 'critical']
    high = [sg for sg in guide.safety_guidelines if sg.importance == 'high']
    medium = [sg for sg in guide.safety_guidelines if sg.importance == 'medium']
    
    if critical:
        md_lines.append("### 🚨 Critical Safety Warnings")
        md_lines.append("")
        for sg in critical:
            md_lines.append(f"- **{sg.category.replace('_', ' ').title()}:** {sg.description}")
        md_lines.append("")
    
    if high:
        md_lines.append("### ⚡ Important Precautions")
        md_lines.append("")
        for sg in high:
            md_lines.append(f"- **{sg.category.replace('_', ' ').title()}:** {sg.description}")
        md_lines.append("")
    
    if medium:
        md_lines.append("### ✓ Additional Safety Tips")
        md_lines.append("")
        for sg in medium:
            md_lines.append(f"- {sg.description}")
        md_lines.append("")
    
    # Materials
    md_lines.extend([
        "## 🛒 Materials List",
        "",
        f"**Total Materials Cost:** {format_currency(guide.cost_breakdown.total_materials)}",
        ""
    ])
    
    for i, material in enumerate(guide.materials, 1):
        md_lines.append(f"### {i}. {material.name}")
        md_lines.append("")
        md_lines.append(f"**Purpose:** {material.description}")
        if material.specifications:
            md_lines.append(f"**Specifications:** {material.specifications}")
        if material.estimated_price:
            md_lines.append(f"**Price Range:** {material.estimated_price}")
        md_lines.append("")
        
        if material.retailers:
            md_lines.append("**Where to Buy:**")
            md_lines.append("")
            for retailer in material.retailers:
                product_info = retailer.get('product_name', '')
                price = retailer.get('price', 'N/A')
                notes = retailer.get('notes', '')
                md_lines.append(f"- **{retailer['name']}:** {price}")
                if product_info:
                    md_lines.append(f"  - Product: {product_info}")
                if notes:
                    md_lines.append(f"  - Note: {notes}")
            md_lines.append("")
        
        if material.alternatives:
            md_lines.append("**Alternatives:**")
            for alt in material.alternatives:
                md_lines.append(f"- {alt}")
            md_lines.append("")
    
    # Tools
    md_lines.extend([
        "## 🔧 Tools Required",
        "",
        f"**Total Tools Cost:** {format_currency(guide.cost_breakdown.total_tools)} *(one-time investment)*",
        ""
    ])
    
    for i, tool in enumerate(guide.tools, 1):
        md_lines.append(f"### {i}. {tool.name}")
        md_lines.append("")
        md_lines.append(f"**Use:** {tool.description}")
        if tool.specifications:
            md_lines.append(f"**Specs:** {tool.specifications}")
        if tool.estimated_price:
            md_lines.append(f"**Price Range:** {tool.estimated_price}")
        md_lines.append("")
        
        if tool.retailers:
            md_lines.append("**Where to Buy:**")
            md_lines.append("")
            for retailer in tool.retailers:
                product_info = retailer.get('product_name', '')
                price = retailer.get('price', 'N/A')
                md_lines.append(f"- **{retailer['name']}:** {price}")
                if product_info:
                    md_lines.append(f"  - Product: {product_info}")
            md_lines.append("")
    
    # Cost Breakdown
    md_lines.extend([
        "## 💰 Cost Breakdown",
        "",
        "### Materials",
        ""
    ])
    
    for item, cost in guide.cost_breakdown.materials_cost.items():
        md_lines.append(f"- {item}: {format_currency(cost)}")
    
    md_lines.extend([
        "",
        f"**Subtotal Materials:** {format_currency(guide.cost_breakdown.total_materials)}",
        "",
        "### Tools",
        ""
    ])
    
    for item, cost in guide.cost_breakdown.tools_cost.items():
        md_lines.append(f"- {item}: {format_currency(cost)}")
    
    md_lines.extend([
        "",
        f"**Subtotal Tools:** {format_currency(guide.cost_breakdown.total_tools)}",
        "",
        f"### 🎯 Total Project Cost: {format_currency(guide.cost_breakdown.total_project)}",
        ""
    ])
    
    if guide.cost_breakdown.cost_notes:
        md_lines.append("**Cost Notes:**")
        for note in guide.cost_breakdown.cost_notes:
            md_lines.append(f"- {note}")
        md_lines.append("")
    
    # Preparation
    if guide.preparation_steps:
        md_lines.extend([
            "## 🎯 Preparation",
            ""
        ])
        for i, step in enumerate(guide.preparation_steps, 1):
            md_lines.append(f"{i}. {step}")
        md_lines.append("")
    
    # Step-by-step instructions
    md_lines.extend([
        "## 📖 Step-by-Step Instructions",
        ""
    ])
    
    for step in guide.steps:
        md_lines.append(f"### Step {step.step_number}: {step.title}")
        md_lines.append("")
        md_lines.append(step.description)
        md_lines.append("")
        
        # Image
        if step.image_path:
            md_lines.append(f"![Step {step.step_number}]({step.image_path})")
            md_lines.append("")
        
        # Detailed instructions
        md_lines.append("**Instructions:**")
        md_lines.append("")
        for i, instruction in enumerate(step.detailed_instructions, 1):
            md_lines.append(f"{i}. {instruction}")
        md_lines.append("")
        
        # Required products
        if step.required_products:
            md_lines.append("**Required Products:**")
            for product in step.required_products:
                md_lines.append(f"- {product}")
            md_lines.append("")
        
        # Tips
        if step.tips:
            md_lines.append("**💡 Pro Tips:**")
            for tip in step.tips:
                md_lines.append(f"- {tip}")
            md_lines.append("")
        
        # Warnings
        if step.warnings:
            md_lines.append("**⚠️ Warnings:**")
            for warning in step.warnings:
                md_lines.append(f"- {warning}")
            md_lines.append("")
        
        md_lines.append("---")
        md_lines.append("")
    
    # Cleanup
    if guide.cleanup_instructions:
        md_lines.extend([
            "## 🧹 Cleanup & Disposal",
            "",
            guide.cleanup_instructions,
            ""
        ])
    
    # Maintenance
    if guide.maintenance_tips:
        md_lines.extend([
            "## 🔧 Maintenance Tips",
            ""
        ])
        for tip in guide.maintenance_tips:
            md_lines.append(f"- {tip}")
        md_lines.append("")
    
    # Troubleshooting
    if guide.troubleshooting:
        md_lines.extend([
            "## 🔍 Troubleshooting",
            ""
        ])
        for problem, solution in guide.troubleshooting.items():
            md_lines.append(f"**Problem:** {problem}")
            md_lines.append(f"**Solution:** {solution}")
            md_lines.append("")
    
    # Footer
    md_lines.extend([
        "---",
        "",
        "*This guide was generated using AI and should be used as a reference. Always follow manufacturer instructions and local building codes. Consult professionals when needed.*",
        ""
    ])
    
    # Write to file
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write('\n'.join(md_lines))
    
    print(f'✅ Exported to: {filepath}')
    return str(filepath)

print('✅ Export functions defined')

✅ Export functions defined


## 🎨 Display Functions

In [97]:
def display_guide_summary(guide: CompleteDIYGuide):
    """Display a nice summary of the guide."""
    
    html = f"""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                padding: 20px; border-radius: 10px; color: white; margin: 20px 0;">
        <h2 style="margin-top: 0;">📋 {guide.topic}</h2>
        <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px; margin-top: 15px;">
            <div style="background: rgba(255,255,255,0.1); padding: 10px; border-radius: 5px;">
                <strong>Difficulty:</strong><br>{guide.difficulty}
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 10px; border-radius: 5px;">
                <strong>Time:</strong><br>{guide.duration_estimate}
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 10px; border-radius: 5px;">
                <strong>Total Cost:</strong><br>{format_currency(guide.cost_breakdown.total_project)}
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 10px; border-radius: 5px;">
                <strong>Steps:</strong><br>{len(guide.steps)}
            </div>
        </div>
    </div>
    
    <div style="background: #f8f9fa; padding: 20px; border-radius: 10px; margin: 20px 0;">
        <h3>📊 Cost Breakdown</h3>
        <div style="margin: 10px 0;">
            <strong>Materials:</strong> {format_currency(guide.cost_breakdown.total_materials)}<br>
            <strong>Tools:</strong> {format_currency(guide.cost_breakdown.total_tools)}<br>
        </div>
        <div style="background: #e9ecef; padding: 10px; border-radius: 5px; margin-top: 10px;">
            <strong>Materials Needed:</strong> {len(guide.materials)}<br>
            <strong>Tools Required:</strong> {len(guide.tools)}<br>
            <strong>Safety Guidelines:</strong> {len(guide.safety_guidelines)}
        </div>
    </div>
    """
    
    display(HTML(html))

print('✅ Display functions defined')

✅ Display functions defined


## 🚀 Generate Your DIY Guide

Now let's create a complete DIY guide! Modify the parameters below to customize your guide.

In [98]:
# Configuration
DIY_TOPIC = "How to Install a French Drain"
BUDGET_RANGE = "moderate"  # Options: 'budget', 'moderate', 'premium'
GENERATE_IMAGES = True  # Set to False to skip image generation

# Generate the guide
guide = generate_complete_guide(
    topic=DIY_TOPIC,
    budget_range=BUDGET_RANGE,
    generate_images=GENERATE_IMAGES
)

# Display summary
display_guide_summary(guide)

🔄 Generating complete DIY guide for: How to Install a French Drain
💰 Budget range: moderate
🎨 Generate images: True

📝 Step 1: Generating guide content...


✅ Successfully parsed JSON response
✅ Generated 8 steps with 6 materials and 11 tools

🎨 Step 2: Generating images for 8 steps...
  Generating image 1/8: Establish Trench Layout and Grade...


C:\Users\ramma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\genai\_common.py:473: UserWarning: block_some is not a valid SafetyFilterLevel
  warnings.warn(f"{value} is not a valid {cls.__name__}")


    ⚠️ Failed to generate image: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Only block_low_and_above is supported for 
  Generating image 2/8: Dig the Trench...
    ⚠️ Failed to generate image: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Only block_low_and_above is supported for 
  Generating image 3/8: Lay Geotextile Filter Fabric...
    ⚠️ Failed to generate image: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Only block_low_and_above is supported for 
  Generating image 4/8: Add Initial Gravel Layer...
    ⚠️ Failed to generate image: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Only block_low_and_above is supported for 
  Generating image 5/8: Install Perforated Drain Pipe...
    ⚠️ Failed to generate image: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Only block_low_and_above is supported for 
  Generating image 4/8: Add Initial Gravel Layer...
    ⚠️ Failed to generate image: 400 INVALID_ARGUMENT. {'error': {'code

## 💾 Export Guide to Markdown

In [99]:
# Export the guide
output_file = export_guide_to_markdown(guide)

print(f"\n📄 Guide exported successfully!")
print(f"📍 Location: {output_file}")

✅ Exported to: ..\exports\diy_guides\how_to_install_a_french_drain.md

📄 Guide exported successfully!
📍 Location: ..\exports\diy_guides\how_to_install_a_french_drain.md


## 📊 View Guide Details

In [100]:
# View materials with pricing
print("🛒 MATERIALS LIST WITH PRICING\n" + "="*60)

for i, material in enumerate(guide.materials, 1):
    print(f"\n{i}. {material.name}")
    print(f"   Description: {material.description}")
    if material.specifications:
        print(f"   Specs: {material.specifications}")
    if material.estimated_price:
        print(f"   Price Range: {material.estimated_price}")
    
    if material.retailers:
        print("\n   Where to Buy:")
        for retailer in material.retailers:
            print(f"   - {retailer['name']}: {retailer.get('price', 'N/A')}")
            if retailer.get('product_name'):
                print(f"     Product: {retailer['product_name']}")

🛒 MATERIALS LIST WITH PRICING

1. Perforated Corrugated Drain Pipe with Sock
   Description: The main component of the French drain, designed to collect and transport subsurface water. The sock prevents sediment from entering the pipe.
   Specs: 4-inch diameter, 100-foot roll, perforated with filtering sock attached.
   Price Range: $60.00 - $85.00

   Where to Buy:
   - Home Depot: $64.97
     Product: NDS 4 in. x 100 ft. Corrugated Perforated Drain Pipe with Sock
   - Lowe's: $69.98
     Product: ADS 4-in x 100-ft Corrugated Perforated Drain Pipe with Sock
   - Amazon: $75.00
     Product: Flex-Drain 4 in. x 100 ft. Perforated Corrugated Drain Pipe with Sock

2. Non-Woven Geotextile Filter Fabric
   Description: A heavy-duty fabric that lines the trench, preventing soil and fines from mixing with the gravel and clogging the drain system, while allowing water to pass through.
   Specs: 3-foot wide x 100-foot long roll, 3 oz. per square yard minimum.
   Price Range: $30.00 - $60.00

  

In [101]:
# View tools with pricing
print("🔧 TOOLS REQUIRED WITH PRICING\n" + "="*60)

for i, tool in enumerate(guide.tools, 1):
    print(f"\n{i}. {tool.name}")
    print(f"   Use: {tool.description}")
    if tool.specifications:
        print(f"   Specs: {tool.specifications}")
    if tool.estimated_price:
        print(f"   Price Range: {tool.estimated_price}")
    
    if tool.retailers:
        print("\n   Where to Buy:")
        for retailer in tool.retailers:
            print(f"   - {retailer['name']}: {retailer.get('price', 'N/A')}")
            if retailer.get('product_name'):
                print(f"     Product: {retailer['product_name']}")

🔧 TOOLS REQUIRED WITH PRICING

1. Trenching Shovel (or Ditch Digging Shovel)
   Use: Narrow, pointed shovel designed for digging trenches. Essential for manual digging.
   Specs: Blade width 4-6 inches, long handle.
   Price Range: $25.00 - $45.00

   Where to Buy:
   - Home Depot: $29.98
     Product: Fiskars Pro D-Handle Digging Shovel
   - Lowe's: $32.98
     Product: Kobalt D-Handle Steel Digging Shovel

2. Square Point Shovel
   Use: Used for scooping loose soil and gravel, and for leveling the trench bottom.
   Specs: Standard square blade, long or D-handle.
   Price Range: $20.00 - $40.00

   Where to Buy:
   - Home Depot: $24.97
     Product: Husky 48 in. Wood Handle Square Point Shovel
   - Lowe's: $26.98
     Product: Kobalt Wood Handle Square Point Shovel

3. Pickaxe (or Mattock)
   Use: Crucial for breaking up compacted soil, clay, or small rocks.
   Specs: Standard pickaxe with sharp point and broad blade.
   Price Range: $30.00 - $60.00

   Where to Buy:
   - Home Depot: 

In [102]:
# View safety guidelines
print("⚠️ SAFETY GUIDELINES\n" + "="*60)

critical = [sg for sg in guide.safety_guidelines if sg.importance == 'critical']
high = [sg for sg in guide.safety_guidelines if sg.importance == 'high']
medium = [sg for sg in guide.safety_guidelines if sg.importance == 'medium']

if critical:
    print("\n🚨 CRITICAL WARNINGS:")
    for sg in critical:
        print(f"   - {sg.description}")

if high:
    print("\n⚡ IMPORTANT PRECAUTIONS:")
    for sg in high:
        print(f"   - {sg.description}")

if medium:
    print("\n✓ ADDITIONAL SAFETY TIPS:")
    for sg in medium:
        print(f"   - {sg.description}")

⚠️ SAFETY GUIDELINES

🚨 CRITICAL WARNINGS:
   - Before any digging, ALWAYS call 811 (Dig Safe) at least 3 business days in advance to have underground utility lines (electrical, gas, water, sewer, communication) marked. Hitting a utility line can cause serious injury, death, or massive property damage.
   - Heavy-duty work gloves (e.g., leather or nitrile-dipped) to protect hands from blisters, cuts, and abrasions.
   - ANSI-approved safety glasses or goggles to protect eyes from flying debris, dust, and splashes.
   - Steel-toe work boots with good ankle support and slip-resistant soles to protect feet from falling objects and provide stable footing.
   - If working in a trench deeper than 4 feet, consider trench shoring or sloping the sides to prevent collapse, especially in unstable soil. Never work alone in a deep trench.
   - If you hit a utility line (gas, electrical, water, sewer), immediately stop work, evacuate the area, and call 911 and your utility company. Do NOT attempt to

In [103]:
# View step-by-step instructions
print("📖 STEP-BY-STEP INSTRUCTIONS\n" + "="*60)

for step in guide.steps:
    print(f"\n{'='*60}")
    print(f"STEP {step.step_number}: {step.title}")
    print(f"{'='*60}")
    print(f"\n{step.description}\n")
    
    print("Instructions:")
    for i, instruction in enumerate(step.detailed_instructions, 1):
        print(f"  {i}. {instruction}")
    
    if step.tips:
        print("\n💡 Pro Tips:")
        for tip in step.tips:
            print(f"  - {tip}")
    
    if step.warnings:
        print("\n⚠️ Warnings:")
        for warning in step.warnings:
            print(f"  - {warning}")
    
    if step.image_path:
        print(f"\n📸 Image saved: {step.image_path}")

📖 STEP-BY-STEP INSTRUCTIONS

STEP 1: Establish Trench Layout and Grade

Accurately mark the trench path and establish the required slope using stakes and string line. This is crucial for effective drainage.

Instructions:
  1. Using spray paint or powdered chalk, mark the exact path of your French drain on the ground. Ensure the path avoids any marked utility lines.
  2. At the starting point (highest elevation), drive a stake into the ground. Tie a string line to this stake at the desired depth for the top of your drain, typically 18-24 inches below the finished grade.
  3. Walk the length of your proposed trench to the discharge point (lowest elevation). Drive another stake at this point. Calculate the total drop needed (e.g., 50 feet * 1/8 inch/foot = 6.25 inches).
  4. Tie the string line to the second stake, ensuring it has the calculated downward slope from the first stake. Use a line level (or a 4-foot level resting on the string) to verify the consistent slope along the entire 

## 🎯 Quick Generate Function

Use this cell to quickly generate guides for different topics

In [104]:
def quick_generate(topic: str, budget: str = 'moderate', images: bool = True):
    """Quickly generate and export a DIY guide."""
    print(f"\n🚀 Generating guide for: {topic}\n")
    
    # Generate
    guide = generate_complete_guide(topic, budget, images)
    
    # Display
    display_guide_summary(guide)
    
    # Export
    output_file = export_guide_to_markdown(guide)
    
    print(f"\n✅ Complete! Guide saved to: {output_file}")
    return guide

# Example usage:
# guide = quick_generate("How to Build a Deck", budget='moderate', images=True)
# guide = quick_generate("How to Install Laminate Flooring", budget='budget', images=False)
# guide = quick_generate("How to Tile a Bathroom", budget='premium', images=True)

print("✅ Quick generate function ready!")
print("\nExample usage:")
print('guide = quick_generate("How to Build a Deck")')

✅ Quick generate function ready!

Example usage:
guide = quick_generate("How to Build a Deck")


## 📝 Additional Examples

Uncomment any of these to generate more guides:

In [105]:
# Example 1: Install a Ceiling Fan
guide = quick_generate("How to Install a Ceiling Fan", budget='moderate', images=True)

# Example 2: Build a Raised Garden Bed
# guide = quick_generate("How to Build a Raised Garden Bed", budget='budget', images=True)

# Example 3: Install Recessed Lighting
# guide = quick_generate("How to Install Recessed Lighting", budget='moderate', images=True)

# Example 4: Build a Shed
# guide = quick_generate("How to Build a Storage Shed", budget='premium', images=True)

# Example 5: Install Vinyl Plank Flooring
# guide = quick_generate("How to Install Vinyl Plank Flooring", budget='moderate', images=True)

print("Uncomment any example above to generate a guide!")


🚀 Generating guide for: How to Install a Ceiling Fan

🔄 Generating complete DIY guide for: How to Install a Ceiling Fan
💰 Budget range: moderate
🎨 Generate images: True

📝 Step 1: Generating guide content...
✅ Successfully parsed JSON response
✅ Generated 9 steps with 7 materials and 9 tools

🎨 Step 2: Generating images for 9 steps...
  Generating image 1/9: Disconnect Power and Remove Old Fixture...
✅ Successfully parsed JSON response
✅ Generated 9 steps with 7 materials and 9 tools

🎨 Step 2: Generating images for 9 steps...
  Generating image 1/9: Disconnect Power and Remove Old Fixture...
    ⚠️ Failed to generate image: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Only block_low_and_above is supported for 
  Generating image 2/9: Inspect and Prepare Electrical Box...
    ⚠️ Failed to generate image: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Only block_low_and_above is supported for 
  Generating image 3/9: Assemble Fan Components (Pre-Installation)...

✅ Exported to: ..\exports\diy_guides\how_to_install_a_ceiling_fan.md

✅ Complete! Guide saved to: ..\exports\diy_guides\how_to_install_a_ceiling_fan.md
Uncomment any example above to generate a guide!
